# Classification wine

In [2]:
import pandas as pd
import numpy as np

In [3]:
pd.set_option('max_columns', None)

In [9]:
df_wines = pd.read_csv('df_wines_toClassification.csv', usecols=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'type'])
df_wines.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,White
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,White
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,White
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,White
4,6.2,0.32,0.16,7.0,0.045,30.0,136.0,0.9949,3.18,0.47,White


Segun la conclusion del analisis de los datos, para la clasificacion del vino segun su tipo, excluimos el alcohol y quality.

In [10]:
df_wines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5302 entries, 0 to 5301
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         5302 non-null   float64
 1   volatile acidity      5302 non-null   float64
 2   citric acid           5302 non-null   float64
 3   residual sugar        5302 non-null   float64
 4   chlorides             5302 non-null   float64
 5   free sulfur dioxide   5302 non-null   float64
 6   total sulfur dioxide  5302 non-null   float64
 7   density               5302 non-null   float64
 8   pH                    5302 non-null   float64
 9   sulphates             5302 non-null   float64
 10  type                  5302 non-null   object 
dtypes: float64(10), object(1)
memory usage: 455.8+ KB


#### Cambiaremos los valores de type, 0-> Red, 1 -> White

In [11]:
from sklearn.preprocessing import LabelEncoder

In [12]:
le = LabelEncoder()
df_wines['type_encode'] = le.fit_transform(df_wines['type'])

In [16]:
df_wines[['type_encode','type']].head(2)

,type_encode,type
0,1,White
1,1,White


In [15]:
df_wines[['type_encode','type']].tail(2)

,type_encode,type
5300,0,Red
5301,0,Red


In [18]:
df_wines.type_encode.unique()

array([1, 0])

In [19]:
from sklearn.preprocessing import StandardScaler

In [23]:
scaler = StandardScaler()
df_wines[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates']] = scaler.fit_transform(df_wines[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates']])

In [24]:
df_wines

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
0,-0.159701,-0.440838,0.292387,3.553645,-0.324804,0.863307,0.998873,2.238849,-1.405957,-0.565671,White,1
1,-0.693989,-0.261561,0.154343,-0.778445,-0.211410,-0.917994,0.323077,-0.180141,0.467977,-0.290775,White,1
2,0.679896,-0.381079,0.568475,0.423653,-0.183062,0.001387,-0.299366,0.199986,0.218119,-0.634395,White,1
3,-0.007047,-0.679873,0.016299,0.786551,0.043726,0.978230,1.283418,0.372771,-0.219132,-0.909291,White,1
4,-0.770316,-0.142043,-1.088053,0.446335,-0.324804,0.001387,0.394213,0.130872,-0.281597,-0.428223,White,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5297,-0.312355,1.650725,-1.640229,-0.710401,0.327210,-0.113535,-1.348628,0.687240,1.217550,1.977120,Red,0
5298,-0.770316,1.531207,-1.640229,-0.687720,0.950876,0.116310,-1.241923,0.130872,1.404944,0.327742,Red,0
5299,-0.999297,1.232412,-1.502185,-0.642358,0.157119,0.518539,-1.117434,0.206897,1.842195,1.564775,Red,0
5300,-0.999297,1.800122,-1.364141,-0.687720,0.525649,0.116310,-1.241923,0.327847,2.154517,1.221155,Red,0


Con los datos estandarizados, procedemos a implementar algoritmos de clasificacion pero antes:

In [26]:
df_wines.shape

(5302, 12)

In [30]:
df_wines_red = df_wines[df_wines['type']== 'Red']

In [29]:
df_wines[df_wines['type']== 'White'].shape

(3951, 12)

La idea es utilizar un dataset de entrenamiento balanceado segun las clases del tipo de vino. Por esa razon aleatoriamente escogeremos 1351 datos de vino blanco igualando la cantidad de datos que tiene el vino rojo.

In [31]:
df_wines_white = df_wines[df_wines['type']== 'White'].sample(1351)


In [32]:
df_wines_white.shape

(1351, 12)

In [34]:
df_wines_class = pd.concat([df_wines_white,df_wines_red], ignore_index= True)
df_wines_class.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
0,1.061530,0.096993,-0.397833,1.376259,-0.750031,0.777115,1.016657,0.096315,-1.218563,-0.290775,White,1
1,-0.312355,-0.022525,0.292387,0.877275,-0.778379,0.805846,0.251941,-0.463508,0.343048,-1.252912,White,1
2,0.756223,-0.978668,0.430431,-0.891850,-0.466546,0.633462,-0.388287,-1.348167,-2.155530,0.465190,White,1
3,-0.388681,-0.500597,-0.190767,0.469016,-0.579940,-0.113535,-0.246014,0.282923,-0.906241,-1.184188,White,1
4,1.672146,-1.098186,-0.121745,-0.823807,-0.409849,-0.228458,1.105577,0.303657,-0.468990,-0.496947,White,1


In [37]:
df_wines_class = df_wines_class.sample(frac=1)

In [38]:
df_wines_class.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
2299,2.664396,0.336028,0.361409,-0.710401,0.695740,-0.745610,-1.508684,0.766721,-0.656383,0.808811,Red,0
482,-0.999297,-0.022525,-0.121745,-0.279460,-0.608288,1.552843,0.376429,-1.507130,-0.843777,-1.321636,White,1
383,-0.846643,0.096993,-0.397833,0.707167,-0.608288,-0.917994,-0.459423,-1.455294,-1.031170,-1.802704,White,1
53,-0.312355,-0.022525,2.915222,-0.506271,0.894179,-0.400842,1.265634,-0.594825,1.779730,1.152431,White,1
2234,1.748473,0.276270,0.637497,0.877275,0.355558,-0.688149,-1.330843,1.406025,0.405512,1.908396,Red,0


In [39]:
from sklearn.model_selection import train_test_split

In [40]:
X = df_wines_class.drop(['type','type_encode'], axis=1)
y = df_wines_class['type_encode']

In [41]:
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2, stratify=y)

In [42]:
from sklearn.metrics import accuracy_score

In [51]:
from sklearn.metrics import confusion_matrix

In [44]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [45]:
clf1 = KNeighborsClassifier()
clf2 = LogisticRegression()
clf3 = SVC()
clf4 = DecisionTreeClassifier()
clf5 = RandomForestClassifier()

In [46]:
clf1.fit(x_train,y_train)
y_pred_clf1 = clf1.predict(x_test)
print("Metrica: ", accuracy_score(y_pred_clf1,y_test))

Metrica:  0.988909426987061


In [52]:
confusion_matrix(y_test,y_pred_clf1)

array([[267,   3],
       [  3, 268]], dtype=int64)

In [47]:
clf2.fit(x_train,y_train)
y_pred_clf2 = clf2.predict(x_test)
print("Metrica: ", accuracy_score(y_pred_clf2,y_test))

Metrica:  0.9907578558225508


In [53]:
confusion_matrix(y_test,y_pred_clf2)

array([[268,   2],
       [  3, 268]], dtype=int64)

In [48]:
clf3.fit(x_train,y_train)
y_pred_clf3 = clf3.predict(x_test)
print("Metrica: ", accuracy_score(y_pred_clf3,y_test))

Metrica:  0.9926062846580407


In [54]:
confusion_matrix(y_test,y_pred_clf3)

array([[268,   2],
       [  2, 269]], dtype=int64)

In [49]:
clf4.fit(x_train,y_train)
y_pred_clf4 = clf4.predict(x_test)
print("Metrica: ", accuracy_score(y_pred_clf4,y_test))

Metrica:  0.977818853974122


In [55]:
confusion_matrix(y_test,y_pred_clf4)

array([[260,  10],
       [  2, 269]], dtype=int64)

In [50]:
clf5.fit(x_train,y_train)
y_pred_clf5 = clf5.predict(x_test)
print("Metrica: ", accuracy_score(y_pred_clf5,y_test))

Metrica:  0.9907578558225508


In [56]:
confusion_matrix(y_test,y_pred_clf5)

array([[266,   4],
       [  1, 270]], dtype=int64)

## PRUEBA FINAL

In [65]:
r_5 = df_wines[df_wines['type'] == 'Red'].sample(5)
w_5 = df_wines[df_wines['type'] =='White'].sample(5)

In [66]:
r_5

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
4104,0.221934,1.053136,0.706518,-0.619677,0.865830,-1.262762,-1.348628,0.925683,2.216982,0.533914,Red,0
4587,0.832549,2.487350,-0.190767,-0.188736,0.525649,-1.032917,-1.739878,0.683784,1.404944,1.014983,Red,0
5172,-0.007047,0.216511,-0.052723,-0.687720,-0.012971,-0.860533,-1.508684,0.068669,0.030726,1.564775,Red,0
4575,0.679896,2.248314,-1.571207,-0.506271,0.780785,-0.688149,-1.153003,1.685937,1.280015,1.289879,Red,0
4274,0.145607,1.232412,-0.673921,-0.642358,1.404451,-1.032917,-0.743968,0.476442,-1.093635,0.671362,Red,0


In [69]:
rw_10 = pd.concat([r_5,w_5],ignore_index=True)
rw_10

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
0,0.221934,1.053136,0.706518,-0.619677,0.865830,-1.262762,-1.348628,0.925683,2.216982,0.533914,Red,0
1,0.832549,2.487350,-0.190767,-0.188736,0.525649,-1.032917,-1.739878,0.683784,1.404944,1.014983,Red,0
2,-0.007047,0.216511,-0.052723,-0.687720,-0.012971,-0.860533,-1.508684,0.068669,0.030726,1.564775,Red,0
3,0.679896,2.248314,-1.571207,-0.506271,0.780785,-0.688149,-1.153003,1.685937,1.280015,1.289879,Red,0
4,0.145607,1.232412,-0.673921,-0.642358,1.404451,-1.032917,-0.743968,0.476442,-1.093635,0.671362,Red,0
5,-0.236028,-0.082284,-0.259789,-0.846488,-0.154713,0.403617,1.301202,-0.629382,0.280583,0.465190,White,1
6,-1.228278,0.873859,-1.295119,-0.120693,-0.494895,-0.745610,0.038532,-1.320522,1.217550,2.526912,White,1
7,-0.388681,-0.620114,0.982606,-0.642358,-0.664985,-0.630687,-0.050389,-1.406914,-0.781312,0.602638,White,1
8,0.221934,-0.978668,0.361409,0.264886,-0.183062,-0.515765,0.429782,0.324391,-0.156668,0.121570,White,1
9,-0.770316,-0.321320,0.016299,-0.324823,-0.863424,0.518539,0.429782,-1.838877,0.530441,-1.115464,White,1


In [71]:
rw_10 = rw_10.sample(frac=1)

In [84]:
rw_10

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,type,type_encode
9,-0.770316,-0.321320,0.016299,-0.324823,-0.863424,0.518539,0.429782,-1.838877,0.530441,-1.115464,White,1
4,0.145607,1.232412,-0.673921,-0.642358,1.404451,-1.032917,-0.743968,0.476442,-1.093635,0.671362,Red,0
1,0.832549,2.487350,-0.190767,-0.188736,0.525649,-1.032917,-1.739878,0.683784,1.404944,1.014983,Red,0
3,0.679896,2.248314,-1.571207,-0.506271,0.780785,-0.688149,-1.153003,1.685937,1.280015,1.289879,Red,0
6,-1.228278,0.873859,-1.295119,-0.120693,-0.494895,-0.745610,0.038532,-1.320522,1.217550,2.526912,White,1
8,0.221934,-0.978668,0.361409,0.264886,-0.183062,-0.515765,0.429782,0.324391,-0.156668,0.121570,White,1
0,0.221934,1.053136,0.706518,-0.619677,0.865830,-1.262762,-1.348628,0.925683,2.216982,0.533914,Red,0
2,-0.007047,0.216511,-0.052723,-0.687720,-0.012971,-0.860533,-1.508684,0.068669,0.030726,1.564775,Red,0
7,-0.388681,-0.620114,0.982606,-0.642358,-0.664985,-0.630687,-0.050389,-1.406914,-0.781312,0.602638,White,1
5,-0.236028,-0.082284,-0.259789,-0.846488,-0.154713,0.403617,1.301202,-0.629382,0.280583,0.465190,White,1


In [85]:
rw_10_x = rw_10.drop(['type', 'type_encode'],axis=1)
rw_10_Y = rw_10['type_encode']

In [86]:
rw_10_Y

9    1
4    0
1    0
3    0
6    1
8    1
0    0
2    0
7    1
5    1
Name: type_encode, dtype: int32

In [87]:
y_pred_rw10 = clf3.predict(rw_10_x)

In [88]:
y_prueba = pd.Series(y_pred_rw10)

In [90]:
accuracy_score(rw_10_Y,y_prueba)

1.0

In [91]:
y_pred_rw10 = clf2.predict(rw_10_x)

In [92]:
y_prueba = pd.Series(y_pred_rw10)
accuracy_score(rw_10_Y,y_prueba)

1.0

In [93]:
y_pred_rw10 = clf1.predict(rw_10_x)
y_prueba = pd.Series(y_pred_rw10)
accuracy_score(rw_10_Y,y_prueba)

0.9

In [94]:
y_pred_rw10 = clf4.predict(rw_10_x)
y_prueba = pd.Series(y_pred_rw10)
accuracy_score(rw_10_Y,y_prueba)

1.0

In [95]:
y_pred_rw10 = clf5.predict(rw_10_x)
y_prueba = pd.Series(y_pred_rw10)
accuracy_score(rw_10_Y,y_prueba)

1.0

#### Conclusion

Se probaron diferentes algoritmos de clasificacion para encontrar el tipo de vino segun sus caracteristicas. En orden de mejor a menor encontramos:

* SVM
* Logistic Regression
* KNN
* Random Forest 
* Decision Tree

De igual forma en la prueba con nuevos datos aleatorios. Encontramos que SVM, LogisticRegression, RandomForest y Decision Tree tuvieron una clasificacion 10/10, mientras que KNN obtuvo 9/10 en su clasificacion.